In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import  f1_score
from charformer_pytorch import GBST

torch.Size([1, 232, 512])

In [ ]:
!pip install charformer-pytorch

In [2]:
import random
import numpy as np
import torch 
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [3]:
def eval_conversation(text):
    input_ids = torch.tensor([list(text.encode("utf-8"))]) + 3 # add 3 for special tokens
    return input_ids


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
class MetNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.dim = 232
        self.tokenizer = GBST(
            num_tokens = 257,             # number of tokens, should be 256 for byte encoding (+ 1 special token for padding in this example)
            dim = 512,                    # dimension of token and intra-block positional embedding
            max_block_size = 4,           # maximum block size
            downsample_factor = 4,        # the final downsample factor by which the sequence length will decrease by
            score_consensus_attn = True   # whether to do the cheap score consensus (aka attention) as in eq. 5 in the paper
        ).to(device)
        self.positionEmbeddings = nn.Embedding(512 ,64)
        self.transformerLayer = nn.TransformerEncoderLayer(576,8) 
        self.linear1 = nn.Linear(576,  64)
        self.linear2 = nn.Linear(64,  1)
        self.linear3 = nn.Linear(232,  16)
        self.linear4 = nn.Linear(16,  1)

    def forward(self, x):
        x = x.type(torch.LongTensor).to(device)
        tokenize = self.tokenizer(x)
        positions = (torch.arange(0,self.dim ).reshape(1,self.dim ) + torch.zeros(x.shape[0],self.dim )).to(device)
        sentence = torch.cat((tokenize[0],self.positionEmbeddings(positions.long())),axis=2)
        attended = self.transformerLayer(sentence)
        linear1 = F.relu(self.linear1(attended))
        linear2 = F.relu(self.linear2(linear1))
        linear2 = linear2.view(-1,232) # reshaping the layer as the transformer outputs a 2d tensor (or 3d considering the batch size)
        linear3 = F.relu(self.linear3(linear2))
        out = torch.sigmoid(self.linear4(linear3))
        return out
    

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MetNet().to(device)
tokens = torch.randint(0, 257, (1, 927))# uneven number of tokens (1023)
print(tokens.shape)
print(model(tokens.to(device)).shape)

torch.Size([1, 927])
torch.Size([1, 1])


In [7]:
import numpy as np
import pandas as pd
train=pd.read_json('Sarcasm_Headlines_Dataset_v2.json',lines=True)
train.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [8]:
train=train.drop(['article_link'],axis=1)
train.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


In [9]:
import random

train = train.sample(frac = 1)
train_data = train[:17171]
test_data = train[17171:22895]
val_data = train[22895:]



In [10]:
X_train=list(train_data['headline'])

y_train=list(train_data['is_sarcastic'])
len(X_train)

17171

In [11]:
X_val=list(val_data['headline'])

y_val=list(val_data['is_sarcastic'])
len(X_val)

5724

In [12]:
X_test=list(test_data['headline'])

y_test=list(test_data['is_sarcastic'])


In [13]:
train_data=[]
mi=10000000000000000
ma=-1
for i in range(0,len(X_train)):
    train_data.append(eval_conversation(X_train[i]));
    mi=min(train_data[i].shape[1],mi)
    ma=max(train_data[i].shape[1],ma)

new_train_data=[]

for i in range(0,len(train_data)):
    z = torch.zeros(1, 926-train_data[i].shape[1])
    s = torch.cat((train_data[i],z),1)
    new_train_data.append(s)

new_train_data = torch.stack(new_train_data)

y_train=torch.Tensor(y_train)

new_train_data = torch.squeeze(new_train_data)
print(new_train_data.shape)


torch.Size([17171, 926])


In [14]:
val_data=[]
mi=10000000000000000
ma=-1
for i in range(0,len(X_val)):
    val_data.append(eval_conversation(X_val[i]));
    mi=min(val_data[i].shape[1],mi)
    ma=max(val_data[i].shape[1],ma)

new_val_data=[]

for i in range(0,len(val_data)):
    z = torch.zeros(1, 926-val_data[i].shape[1])
    s = torch.cat((val_data[i],z),1)
    new_val_data.append(s)

new_val_data = torch.stack(new_val_data)

y_val=torch.Tensor(y_val)

new_val_data = torch.squeeze(new_val_data)
print(new_val_data.shape)

torch.Size([5724, 926])


In [15]:
import torch.optim as  optim

In [16]:
import torch.utils.data as data_utils
train=data_utils.TensorDataset(new_train_data ,y_train)


train_loader = torch.utils.data.DataLoader(train, batch_size=32,
               shuffle=True, num_workers=0, pin_memory=True)

In [17]:
import torch.utils.data as data_utils
val=data_utils.TensorDataset(new_val_data ,y_val)


val_loader = torch.utils.data.DataLoader(val, batch_size=32,
               shuffle=True, num_workers=0, pin_memory=True)

In [18]:
lr =  0.0045
optimizer = torch.optim.SGD(model.parameters(), lr,
                            momentum=0.9,
                            weight_decay=1e-4)

In [19]:
def calculateMetrics(ypred,ytrue):
    acc  = accuracy_score(ytrue,ypred)
    f1  = f1_score(ytrue,ypred)
    f1_average  = f1_score(ytrue,ypred,average="macro")
    return " f1 score: "+str(round(f1,3))+" f1 average: "+str(round(f1_average,3))+" accuracy: "+str(round(acc,3))

In [20]:
def train(train_loader, model, optimizer, epoch):

    model.train()
    trainpreds = torch.tensor([])
    traintrues = torch.tensor([])
    valpreds = torch.tensor([])
    valtrues = torch.tensor([])
    for i, (input, target) in enumerate(train_loader):
        input_var = input.cuda()
        target_var = target.cuda()
        optimizer.zero_grad()
        output = model(input_var)
        output = torch.squeeze(output, 1)
        trainpreds = torch.cat((trainpreds,output.cpu().detach()))
        traintrues = torch.cat((traintrues,target_var.cpu().detach()))
        
        loss = F.binary_cross_entropy(output,target_var)
        loss.backward()
        optimizer.step()
        
    err = F.binary_cross_entropy(trainpreds,traintrues)    
    print("train BCE loss: ",err.item(),calculateMetrics(torch.round(trainpreds).numpy(),traintrues.numpy()))
    
    model.eval()
    for i, (input, target) in enumerate(val_loader):
        input_var = input.cuda()
        target_var = target.cuda()
        optimizer.zero_grad()
        output = model(input_var)
        output = torch.squeeze(output, 1)

        valpreds = torch.cat((valpreds,output.cpu().detach()))
        valtrues = torch.cat((valtrues,target_var.cpu().detach()))

        loss = F.binary_cross_entropy(output,target_var)

    print("test BCE loss: ",calculateMetrics(torch.round(valpreds).numpy(),valtrues.numpy()))



In [21]:
from tqdm import tqdm
for epoch in tqdm(range(0, 100)):
    lr = lr * (0.1 ** (epoch // 100))
    train(train_loader, model, optimizer, epoch)

  0%|          | 0/100 [00:00<?, ?it/s]

train BCE loss:  0.6919268369674683  f1 score: 0.0 f1 average: 0.345 accuracy: 0.526


  1%|          | 1/100 [00:30<50:46, 30.77s/it]

test BCE loss:   f1 score: 0.0 f1 average: 0.343 accuracy: 0.521
train BCE loss:  0.6914328336715698  f1 score: 0.0 f1 average: 0.345 accuracy: 0.526


  2%|▏         | 2/100 [01:01<50:15, 30.77s/it]

...

test BCE loss:   f1 score: 0.772 f1 average: 0.765 accuracy: 0.765
train BCE loss:  0.032881442457437515  f1 score: 0.987 f1 average: 0.988 accuracy: 0.988


 99%|█████████▉| 99/100 [50:58<00:30, 30.90s/it]

test BCE loss:   f1 score: 0.773 f1 average: 0.774 accuracy: 0.774
train BCE loss:  0.025012295693159103  f1 score: 0.99 f1 average: 0.99 accuracy: 0.99


100%|██████████| 100/100 [51:29<00:00, 30.89s/it]

test BCE loss:   f1 score: 0.757 f1 average: 0.776 accuracy: 0.777


In [23]:
torch.save(model.state_dict(), "charformer-100.pt")

In [ ]:
!nvidia-smi

In [26]:
test_data=[]
mi=10000000000000000
ma=-1
for i in range(0,len(X_test)):
    test_data.append(eval_conversation(X_test[i]));
    mi=min(test_data[i].shape[1],mi)
    ma=max(test_data[i].shape[1],ma)

print(mi,ma)
new_test_data=[]

for i in range(0,len(test_data)):
    z = torch.zeros(1, 926-test_data[i].shape[1])
    s = torch.cat((test_data[i],z),1)
    new_test_data.append(s)

new_test_data = torch.stack(new_test_data)

y_test=torch.Tensor(y_test)

new_test_data = torch.squeeze(new_test_data)
print(new_test_data.shape)

9 228
torch.Size([5724, 926])


In [27]:
model = MetNet().to(device)
model.load_state_dict(torch.load("charformer-100.pt"))
model.eval()

MetNet(
  (tokenizer): GBST(
    (token_emb): Embedding(257, 512)
    (pos_conv): Sequential(
      (0): Pad()
      (1): Rearrange('b n d -> b d n')
      (2): DepthwiseConv1d(
        (conv): Conv1d(512, 512, kernel_size=(4,), stride=(1,), groups=512)
        (proj_out): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
      )
      (3): Rearrange('b d n -> b n d')
    )
    (score_fn): Sequential(
      (0): Linear(in_features=512, out_features=1, bias=True)
      (1): Rearrange('... () -> ...')
    )
  )
  (positionEmbeddings): Embedding(512, 64)
  (transformerLayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=576, out_features=576, bias=True)
    )
    (linear1): Linear(in_features=576, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=576, bias=True)
    (norm1): LayerNorm((576,), eps=1e-05, elementwise_affine=True)
    (nor

In [28]:
import torch.utils.data as data_utils
test=data_utils.TensorDataset(new_test_data ,y_val)


test_loader = torch.utils.data.DataLoader(test, batch_size=32,
               shuffle=True, num_workers=0, pin_memory=True)

In [30]:
testpreds = torch.tensor([])
testtrues = torch.tensor([])
for i, (input, target) in enumerate(val_loader):
    input_var = input.cuda()
    target_var = target.cuda()
    optimizer.zero_grad()
    output = model(input_var)
    output = torch.squeeze(output, 1)

    testpreds = torch.cat((testpreds,output.cpu().detach()))
    testtrues = torch.cat((testtrues,target_var.cpu().detach()))

    loss = F.binary_cross_entropy(output,target_var)

print("test BCE loss: ",calculateMetrics(torch.round(testpreds).numpy(),testtrues.numpy()))   

test BCE loss:   f1 score: 0.756 f1 average: 0.775 accuracy: 0.776
